In [1]:
from bs4 import BeautifulSoup
import requests
import re

Airline Data

In [2]:
url ='https://www.airlinequality.com/airline-reviews/british-airways'
page= requests.get(url)
soup= BeautifulSoup(page.content,"html.parser")

In [3]:
rating, header,author, date, place, verification, content, collect=[],[],[],[],[],[],[],[]
urls=[url]
c=1
while(soup!=None):
    articles = soup.find_all("article", attrs={'class':re.compile('comp comp_media-review-rated list-item media position-content review-[0-9]*')})
    for article in articles:
        if(article.find("div",class_='rating-10').find('span') is None):
            rating.append(None)
        else:
            rating.append(article.find("div",class_='rating-10').find('span').text)
        if(article.find("div",class_='body').find('h2') is None):
            header.append(None)
        else:
            header.append(article.find("div",class_='body').find('h2').text.strip("\""))
        if(article.find("div",class_='body').find('span',itemprop='author').find('span') is None):
            author.append(None)
        else:
            author.append(article.find("div",class_='body').find('span',itemprop='author').find('span').text)
        if(article.find("div",class_='body').find('time',itemprop='datePublished') is None):
            date.append(None)
        else:
            date.append(article.find("div",class_='body').find('time',itemprop='datePublished')['datetime'])
        if(re.search(r".*(\(.*\)).*",article.find("div",class_='body').find('h3').text.strip()) is None):
            place.append(None)
        else: 
            place.append(re.search(r".*(\(.*\)).*",article.find("div",class_='body').find('h3').text.strip()).group(1).strip('()'))
        content.append(article.find("div",class_='text_content').text.strip())
        tr= article.find("div",class_='body').find('table', class_='review-ratings').findAll(lambda tag: tag.name=='tr')
        table_dict={}
        for row in tr:
            key= row.find('td',attrs={'class':re.compile('review-rating-header.*')}).text
            if(row.find('td',class_='review-value')==None):
                if(not [i.text for i in row.find('td',attrs={'class':re.compile('review-rating-stars.*')}).findAll('span',class_='star fill')]):
                    value=None
                else:
                    value=max([i.text for i in row.find('td',attrs={'class':re.compile('review-rating-stars.*')}).findAll('span',class_='star fill')])
            else:
                value= row.find('td',class_='review-value').text
            table_dict[key]=value
        collect.append(table_dict)
    next_article=soup.find('article', class_='comp comp_reviews-pagination querylist-pagination position-')
    c+=1
    next_url="https://www.airlinequality.com"+"/airline-reviews/british-airways/page/"+str(c)+"/"
    urls.append(next_url)
    page= requests.get(next_url)
    soup= BeautifulSoup(page.content,"html.parser")
    if(c==367):
        break

In [4]:
import pandas as pd
import numpy as np

In [5]:
airline_review = pd.DataFrame({"rating":rating, 
                               "header":header, 
                               "author":author, 
                               "date":date,
                               "place":place,
                               "content":content, 
                               "collect":collect})

In [6]:
airline_review.dropna(inplace=True)

In [7]:
airline_review.rating=airline_review.rating.astype(int)

In [8]:
airline_review.date=pd.to_datetime(airline_review.date)

In [29]:
new_airline_review=airline_review.merge(pd.json_normalize(airline_review.collect).rename(columns={
    'Aircraft':'aircraft',
    'Type Of Traveller':'traveller_type',
    'Seat Type':'seat_type',
    'Route':'route',
    'Date Flown':'date_flown',
    'Seat Comfort':'seat_comfort',
    'Cabin Staff Service':'cabin_staff_service', 
    'Food & Beverages':'food_beverages',
    'Ground Service':'ground_service',
    'Value For Money':'value_for_money',
    'Recommended':'recommended',
    'Wifi & Connectivity':'wifi_connectivity',
    'Inflight Entertainment':'entertainment'    
}), left_index=True, right_index=True).drop(columns=['collect'])

In [33]:
new_airline_review.content = new_airline_review.content.str.split('|').apply(lambda x: x[1] if len(x)>=2 else x[0])

In [34]:
new_airline_review.head()

,rating,header,author,date,place,content,aircraft,traveller_type,seat_type,route,date_flown,seat_comfort,cabin_staff_service,food_beverages,ground_service,value_for_money,recommended,wifi_connectivity,entertainment
0,2,service was mediocre at best,Gary Storer,2023-10-03,United Kingdom,"Just returned from Chicago, flew out 10 days ...",A380,Couple Leisure,Economy Class,Chicago to Manchester via Heathrow,October 2023,2,3,1,2,2,no,NaN,NaN
1,2,BA standards continue to decline,A Jensen,2023-10-02,United Kingdom,BA standards continue to decline every time ...,A320,Business,Business Class,London Heathrow to Munich,September 2023,2,1,2,1,1,no,1,NaN
2,2,"won the race to the bottom""",John Rockett,2023-10-02,United Kingdom,Awful. Business class check in queue just as...,A320,Couple Leisure,Business Class,Heathrow to Istanbul,September 2023,2,3,2,1,1,no,NaN,NaN
3,3,Not a reliable airline,Tatiana Bobrovskaya,2023-10-02,United Kingdom,Not a reliable airline. You cannot trust the...,A320,Business,Economy Class,London to Geneva,October 2023,4,4,2,1,1,no,NaN,NaN
4,1,It is a national disgrace,A Dawson,2023-09-30,United Kingdom,I take comfort in reading the last ten or so...,NaN,Couple Leisure,Business Class,Athens to London,September 2023,1,1,1,1,1,no,1,1


In [35]:
new_airline_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3648 entries, 0 to 3653
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   rating               3648 non-null   int32         
 1   header               3648 non-null   object        
 2   author               3648 non-null   object        
 3   date                 3648 non-null   datetime64[ns]
 4   place                3648 non-null   object        
 5   content              3648 non-null   object        
 6   aircraft             1902 non-null   object        
 7   traveller_type       2895 non-null   object        
 8   seat_type            3646 non-null   object        
 9   route                2891 non-null   object        
 10  date_flown           2888 non-null   object        
 11  seat_comfort         3534 non-null   object        
 12  cabin_staff_service  3523 non-null   object        
 13  food_beverages       3269 non-nul

In [37]:
new_airline_review.rating.mean()

4.728344298245614

In [40]:
new_airline_review['trip_verified']=airline_review.content.str.split('|').apply(lambda x: x[0] if len(x)>=2 else None)

In [43]:
new_airline_review.drop(columns={'wifi_connectivity'}, inplace=True)

In [46]:
new_airline_review.date_flown = pd.to_datetime(new_airline_review.date_flown)

In [49]:
new_airline_review.seat_comfort=new_airline_review.seat_comfort.fillna(-1)
new_airline_review.seat_comfort=new_airline_review.seat_comfort.astype(int)
new_airline_review.cabin_staff_service=new_airline_review.cabin_staff_service.fillna(-1)
new_airline_review.cabin_staff_service=new_airline_review.cabin_staff_service.astype(int)
new_airline_review.food_beverages=new_airline_review.food_beverages.fillna(-1)
new_airline_review.food_beverages=new_airline_review.food_beverages.astype(int)
new_airline_review.ground_service=new_airline_review.ground_service.fillna(-1)
new_airline_review.ground_service=new_airline_review.ground_service.astype(int)
new_airline_review.value_for_money=new_airline_review.value_for_money.fillna(-1)
new_airline_review.value_for_money=new_airline_review.value_for_money.astype(int)
new_airline_review.entertainment=new_airline_review.entertainment.fillna(-1)
new_airline_review.entertainment=new_airline_review.entertainment.astype(int)

In [50]:
new_airline_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3648 entries, 0 to 3653
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   rating               3648 non-null   int32         
 1   header               3648 non-null   object        
 2   author               3648 non-null   object        
 3   date                 3648 non-null   datetime64[ns]
 4   place                3648 non-null   object        
 5   content              3648 non-null   object        
 6   aircraft             1902 non-null   object        
 7   traveller_type       2895 non-null   object        
 8   seat_type            3646 non-null   object        
 9   route                2891 non-null   object        
 10  date_flown           2888 non-null   datetime64[ns]
 11  seat_comfort         3648 non-null   int32         
 12  cabin_staff_service  3648 non-null   int32         
 13  food_beverages       3648 non-nul

In [53]:
new_airline_review.head()

,rating,header,author,date,place,content,aircraft,traveller_type,seat_type,route,date_flown,seat_comfort,cabin_staff_service,food_beverages,ground_service,value_for_money,recommended,entertainment,trip_verified
0,2,service was mediocre at best,Gary Storer,2023-10-03,United Kingdom,"Just returned from Chicago, flew out 10 days ...",A380,Couple Leisure,Economy Class,Chicago to Manchester via Heathrow,2023-10-01,2,3,1,2,2,no,-1,Not Verified
1,2,BA standards continue to decline,A Jensen,2023-10-02,United Kingdom,BA standards continue to decline every time ...,A320,Business,Business Class,London Heathrow to Munich,2023-09-01,2,1,2,1,1,no,-1,✅ Trip Verified
2,2,"won the race to the bottom""",John Rockett,2023-10-02,United Kingdom,Awful. Business class check in queue just as...,A320,Couple Leisure,Business Class,Heathrow to Istanbul,2023-09-01,2,3,2,1,1,no,-1,Not Verified
3,3,Not a reliable airline,Tatiana Bobrovskaya,2023-10-02,United Kingdom,Not a reliable airline. You cannot trust the...,A320,Business,Economy Class,London to Geneva,2023-10-01,4,4,2,1,1,no,-1,✅ Trip Verified
4,1,It is a national disgrace,A Dawson,2023-09-30,United Kingdom,I take comfort in reading the last ten or so...,NaN,Couple Leisure,Business Class,Athens to London,2023-09-01,1,1,1,1,1,no,1,✅ Trip Verified


In [51]:
new_airline_review.rating.mean()

4.728344298245614

In [55]:
new_airline_review.trip_verified.unique()

array(['Not Verified ', '✅ Trip Verified ', '❎ Not Verified ', None,
       '✅ Verified Review ', '❎ Unverified ',
       'We travelled economy from Manchester to Toronto via Heathrow The flight from Manchester to Heathrow and return was very good. The seats had plenty of room and it was easy to fit laptop size briefcases under the seat in front and still have room to stretch your legs. The service from the cabin crew was excellent. The leg from Heathrow to Toronto was not as comfortable but it was acceptable. The seats appeared narrower and my case only just fit under the seat leaving little room to stretch my legs. Whilst the cabin crew forgot requests for drinks they apologised and were very friendly and helpful in other aspects. Unfortunately BA were let down by the attendants on the return leg from Toronto to Heathrow. Whilst the aircraft appeared newer and more comfortable I have never experienced such surly behaviour from the cabin crew. We had seen attendants in the opposite ai

In [69]:
airline_review[airline_review.content.str.contains("We travelled economy from Manchester")]['content'].str.split('|').apply(lambda x:x)

3283    [We travelled economy from Manchester to Toron...
3290    [We travelled economy from Manchester to Toron...
Name: content, dtype: object

In [73]:
new_airline_review.loc[3283,'content']=new_airline_review.loc[3283,'content']+new_airline_review.loc[3283,'trip_verified']
new_airline_review.loc[3290,'content']=new_airline_review.loc[3290,'content']+new_airline_review.loc[3290,'trip_verified']

In [74]:
new_airline_review.loc[3283,'trip_verified']=None
new_airline_review.loc[3290,'trip_verified']=None

In [84]:
new_airline_review.trip_verified.value_counts()

✅ Trip Verified       1129
✅ Verified Review      770
Not Verified           218
❎ Unverified            20
❎ Not Verified           5
Name: trip_verified, dtype: int64

In [85]:
def clean_trip(x):
    if(x=='Not Verified ' or x=='❎ Not Verified ' or x=='❎ Unverified '):
        return 'Not Verified'
    elif(x=='✅ Trip Verified ' or x=='✅ Verified Review '):
        return 'Verified'
    else:
        return None

new_airline_review.trip_verified=new_airline_review.trip_verified.apply(clean_trip)

In [86]:
new_airline_review.columns

Index(['rating', 'header', 'author', 'date', 'place', 'content', 'aircraft',
       'traveller_type', 'seat_type', 'route', 'date_flown', 'seat_comfort',
       'cabin_staff_service', 'food_beverages', 'ground_service',
       'value_for_money', 'recommended', 'entertainment', 'trip_verified'],
      dtype='object')

In [107]:
new_airline_review.drop_duplicates(inplace=True)

In [108]:
new_airline_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3616 entries, 0 to 3653
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   rating               3616 non-null   int32         
 1   header               3616 non-null   object        
 2   author               3616 non-null   object        
 3   date                 3616 non-null   datetime64[ns]
 4   place                3616 non-null   object        
 5   content              3616 non-null   object        
 6   aircraft             1902 non-null   object        
 7   traveller_type       2895 non-null   object        
 8   seat_type            3614 non-null   object        
 9   route                2891 non-null   object        
 10  date_flown           2888 non-null   datetime64[ns]
 11  seat_comfort         3616 non-null   int32         
 12  cabin_staff_service  3616 non-null   int32         
 13  food_beverages       3616 non-nul

In [110]:
new_airline_review.to_csv("E:/BA/airline_review.csv")